# Feature Tokenizer Transformer
Featured in the paper [Revisiting Deep Learning Models for Tabular Data (2021, June)](https://arxiv.org/abs/2106.11959) Feature Tokenizer Transformer is a simple adaptation of the Transformer architecture for the tabular domain. In a nutshell, Feature Tokenizer Transformer transforms all features (categorical and numerical) to embeddings and applies a stack of Transformer layers to the embeddings. Thus, every Transformer layer operates on the feature level of one object.

In this notebook we will be implementing Feature Tokenizer Transformer using TensorFlow 2 from scratch.

In [1]:
import numpy as np#导入处理数据或矩阵的库
import pandas as pd#导入具有数据分析能力和处理工具的库
import matplotlib.pyplot as plt#加载窗口，绘制图形
import tensorflow as tf#导入tensorflow模块
from tensorflow import keras#加载Keras人工神经网络库
from tensorflow.keras import layers as L
from tensorflow_addons.activations import sparsemax
from tensorflow.data import Dataset#引入Dataset数据集类
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler#导入SimpleImputer函数，用于数据预处理中的缺失值处理
from sklearn.pipeline import Pipeline#导入Pipeline包，把一系列的类连成一条流水线
from sklearn.model_selection import StratifiedKFold
import joblib#加载joblib库，提供轻量级管道

pd.options.display.max_columns = 300#设置最大显示的数据框列数为300列

# Data
Loading the train and test csv files into `pandas.DataFrame` and splitting the columns as features and target.

We will be using Stratified K folds as our local cross validation.

In [2]:
data = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv')#读取数据文件
data = data.drop_duplicates(subset=data.columns[1:]).reset_index(drop=True)#去除第二列及往后的所有列的重复行
print(data.shape)#查看数据的行列数
data.head()#返回数据集的前五行数据

(123993, 288)


,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,A0T0G10C0,A0T1G0C9,A0T1G1C8,A0T1G2C7,A0T1G3C6,A0T1G4C5,A0T1G5C4,A0T1G6C3,A0T1G7C2,A0T1G8C1,A0T1G9C0,A0T2G0C8,A0T2G1C7,A0T2G2C6,A0T2G3C5,A0T2G4C4,A0T2G5C3,A0T2G6C2,A0T2G7C1,A0T2G8C0,A0T3G0C7,A0T3G1C6,A0T3G2C5,A0T3G3C4,A0T3G4C3,A0T3G5C2,A0T3G6C1,A0T3G7C0,A0T4G0C6,A0T4G1C5,A0T4G2C4,A0T4G3C3,A0T4G4C2,A0T4G5C1,A0T4G6C0,A0T5G0C5,A0T5G1C4,A0T5G2C3,A0T5G3C2,A0T5G4C1,A0T5G5C0,A0T6G0C4,A0T6G1C3,A0T6G2C2,A0T6G3C1,A0T6G4C0,A0T7G0C3,A0T7G1C2,A0T7G2C1,A0T7G3C0,A0T8G0C2,A0T8G1C1,A0T8G2C0,A0T9G0C1,A0T9G1C0,A0T10G0C0,A1T0G0C9,A1T0G1C8,A1T0G2C7,A1T0G3C6,A1T0G4C5,A1T0G5C4,A1T0G6C3,A1T0G7C2,A1T0G8C1,A1T0G9C0,A1T1G0C8,A1T1G1C7,A1T1G2C6,A1T1G3C5,A1T1G4C4,A1T1G5C3,A1T1G6C2,A1T1G7C1,A1T1G8C0,A1T2G0C7,A1T2G1C6,A1T2G2C5,A1T2G3C4,A1T2G4C3,A1T2G5C2,A1T2G6C1,A1T2G7C0,A1T3G0C6,A1T3G1C5,A1T3G2C4,A1T3G3C3,A1T3G4C2,A1T3G5C1,A1T3G6C0,A1T4G0C5,A1T4G1C4,A1T4G2C3,A1T4G3C2,A1T4G4C1,A1T4G5C0,A1T5G0C4,A1T5G1C3,A1T5G2C2,A1T5G3C1,A1T5G4C0,A1T6G0C3,A1T6G1C2,A1T6G2C1,A1T6G3C0,A1T7G0C2,A1T7G1C1,A1T7G2C0,A1T8G0C1,A1T8G1C0,A1T9G0C0,A2T0G0C8,A2T0G1C7,A2T0G2C6,A2T0G3C5,A2T0G4C4,A2T0G5C3,A2T0G6C2,A2T0G7C1,A2T0G8C0,A2T1G0C7,A2T1G1C6,A2T1G2C5,A2T1G3C4,A2T1G4C3,A2T1G5C2,A2T1G6C1,A2T1G7C0,A2T2G0C6,A2T2G1C5,A2T2G2C4,A2T2G3C3,A2T2G4C2,A2T2G5C1,A2T2G6C0,A2T3G0C5,A2T3G1C4,A2T3G2C3,A2T3G3C2,A2T3G4C1,A2T3G5C0,A2T4G0C4,A2T4G1C3,A2T4G2C2,A2T4G3C1,A2T4G4C0,A2T5G0C3,A2T5G1C2,A2T5G2C1,A2T5G3C0,A2T6G0C2,A2T6G1C1,A2T6G2C0,A2T7G0C1,A2T7G1C0,A2T8G0C0,A3T0G0C7,A3T0G1C6,A3T0G2C5,A3T0G3C4,A3T0G4C3,A3T0G5C2,A3T0G6C1,A3T0G7C0,A3T1G0C6,A3T1G1C5,A3T1G2C4,A3T1G3C3,A3T1G4C2,A3T1G5C1,A3T1G6C0,A3T2G0C5,A3T2G1C4,A3T2G2C3,A3T2G3C2,A3T2G4C1,A3T2G5C0,A3T3G0C4,A3T3G1C3,A3T3G2C2,A3T3G3C1,A3T3G4C0,A3T4G0C3,A3T4G1C2,A3T4G2C1,A3T4G3C0,A3T5G0C2,A3T5G1C1,A3T5G2C0,A3T6G0C1,A3T6G1C0,A3T7G0C0,A4T0G0C6,A4T0G1C5,A4T0G2C4,A4T0G3C3,A4T0G4C2,A4T0G5C1,A4T0G6C0,A4T1G0C5,A4T1G1C4,A4T1G2C3,A4T1G3C2,A4T1G4C1,A4T1G5C0,A4T2G0C4,A4T2G1C3,A4T2G2C2,A4T2G3C1,A4T2G4C0,A4T3G0C3,A4T3G1C2,A4T3G2C1,A4T3G3C0,A4T4G0C2,A4T4G1C1,A4T4G2C0,A4T5G0C1,A4T5G1C0,A4T6G0C0,A5T0G0C5,A5T0G1C4,A5T0G2C3,A5T0G3C2,A5T0G4C1,A5T0G5C0,A5T1G0C4,A5T1G1C3,A5T1G2C2,A5T1G3C1,A5T1G4C0,A5T2G0C3,A5T2G1C2,A5T2G2C1,A5T2G3C0,A5T3G0C2,A5T3G1C1,A5T3G2C0,A5T4G0C1,A5T4G1C0,A5T5G0C0,A6T0G0C4,A6T0G1C3,A6T0G2C2,A6T0G3C1,A6T0G4C0,A6T1G0C3,A6T1G1C2,A6T1G2C1,A6T1G3C0,A6T2G0C2,A6T2G1C1,A6T2G2C0,A6T3G0C1,A6T3G1C0,A6T4G0C0,A7T0G0C3,A7T0G1C2,A7T0G2C1,A7T0G3C0,A7T1G0C2,A7T1G1C1,A7T1G2C0,A7T2G0C1,A7T2G1C0,A7T3G0C0,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,-9.536743e-07,-0.000010,-8.583069e-05,-0.000343,-0.000801,-0.001202,-0.001202,-0.000801,-0.000343,-0.000086,-0.000010,-4.291534e-05,-0.000343,-0.001202,-0.002403,-0.003004,-0.002403,-0.001202,-0.000343,-0.000043,-0.000114,-0.000801,-0.002403,-0.004005,-0.004005,-0.002403,-0.000801,-0.000114,-0.000200,-0.001202,-0.003004,-0.004005,-0.003004,-0.001202,-0.000200,-0.000240,-0.001202,-0.002403,-0.002403,-0.001202,-0.000240,-0.000200,-0.000801,-0.001202,-0.000801,-0.000200,-0.000114,-0.000343,-0.000343,-0.000114,-0.000043,-0.000086,-0.000043,-0.000010,-0.000010,-9.536743e-07,-9.536743e-06,-0.000086,-0.000343,-0.000801,-0.001202,-0.001202,-0.000801,-0.000343,-0.000086,-0.000010,-0.000086,-0.000687,-0.002403,-0.004807,-0.006008,-0.004807,-0.002403,-0.000687,-0.000086,-0.000343,-0.002403,-0.007210,-0.002016,-0.012016,-0.007210,0.017597,0.009657,-0.000801,-0.004807,0.007984,-0.006022,0.007984,-0.004807,-0.000801,-0.001202,0.003992,-0.012016,-0.012016,0.003992,0.008798,-0.001202,-0.004807,0.002790,-0.004807,-0.001202,-0.000801,0.007597,0.007597,-0.000801,-0.000343,-0.000687,-0.000343,-0.000086,-0.000086,-0.000010,-0.000043,-0.000343,-0.001202,-0.002403,-0.003004,-0.002403,-0.001202,-0.000343,-0.000043,-0.000343,-0.002403,-0.007210,-0.012016,-0.002016,-0.007210,-0.002403,-0.000343,0.008798,

In [3]:
test = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv')#读取数据文件
print(test.shape)#查看测试集行列数
X_test = test.drop(['row_id'], axis=1)#删除'row_id'这一列

(100000, 287)


In [4]:
X = data.drop(['row_id', 'target'], axis=1)#删除'row_id'这一列
y = pd.get_dummies(data['target'])#把离散的类别信息转化为onehot编码形式

In [5]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)#实例化一个分层k折交叉验证对象,指定分为5个子集,打乱数据,设置随机种子为0

# Model
**Creating some data utility classes**:

`DataConfig` helps to segregate the features into numeric features and categorical features and maintain a vocabulary for the categorical ones.

`DataLoader` class creates `tf.data.Dataset` objects from `pandas.DataFrame` to ensure efficiency in the input pipeline to the model.

In [6]:
class DataConfig:#数据配置管理
    def __init__(self, numeric_feature_names, categorical_features_with_vocabulary):
        self.NUMERIC_FEATURE_NAMES = numeric_feature_names#数字特征
        self.CATEGORICAL_FEATURES_WITH_VOCABULARY = categorical_features_with_vocabulary
        self.CATEGORICAL_FEATURE_NAMES = list(self.CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())#分类特征
        self.FEATURE_NAMES = self.NUMERIC_FEATURE_NAMES + self.CATEGORICAL_FEATURE_NAMES
        
class DataLoader:#数据加载管理
    @classmethod#类方法
    def from_df(cls, X, y=None, batch_size=1024):
        return (
            Dataset.from_tensor_slices(({col: X[col].values.tolist() for col in X.columns}, y.values.tolist())).batch(
                batch_size#单次训练的数据个数
            )#将多个数据特征进行分割并依次对应组合产生新的配对好的数据
            if y is not None
            else Dataset.from_tensor_slices({col: X[col].values.tolist() for col in X.columns}).batch(batch_size)
        )

**Creating Input Layers and Feature Encoding Layers**

`get_inputs` returns a dictionary of Input Layers based on the data types of the feature columns mentioned in the `DataConfig` object.

`encode_inputs` applies StringLookup and Embedding Layer to the categorical features and Reshapes the Numeric Features in order to encode the inputs.

In [7]:
def get_inputs(config):#输入图层
    return {
        feature_name: L.Input(
            name=feature_name,
            shape=(),
            dtype=(tf.float32 if feature_name in config.NUMERIC_FEATURE_NAMES else tf.string),
        )#返回一个基于DataConfig对象中提到的特征列的数据类型的输入层字典
        for feature_name in config.FEATURE_NAMES
    }

def encode_inputs(inputs, config, use_embeddings=False, embedding_dim=32, prefix="", concat_features=False):#要素编码图层
    cat_features = []#定义训练数据中类别特征的列表
    num_features = []#定义训练数据中数字特征的列表
    for feature_name in inputs:
        if feature_name in config.CATEGORICAL_FEATURE_NAMES:
            vocabulary = config.CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
            lookup = L.StringLookup(
                vocabulary=vocabulary,#词汇
                mask_token=None,#经过mask处理后的分词结果
                num_oov_indices=0,#要使用的词汇表外标记的数量为0，OOV输入将在调用层时导致错误
                output_mode="int" if use_embeddings else "binary",#如果满足条件输出为整数，否则输出为二进制
                name=f"{prefix}{feature_name}_lookup",
            )#应用于分类特征
            if use_embeddings:
                encoded_feature = lookup(inputs[feature_name])#编码特征
                embedding = L.Embedding(
                    input_dim=len(vocabulary),#输入数据的维度取词汇的长度
                    output_dim=embedding_dim,#输出数据的维度取编码的维度
                    name=f"{prefix}{feature_name}_embeddings",
                )
                encoded_feature = embedding(encoded_feature)#将新的编码特征添加到对应列表中
            else:
                encoded_feature = lookup(
                    L.Reshape((1,), name=f"{prefix}{feature_name}_reshape")(inputs[feature_name])
                )
            cat_features.append(encoded_feature)#将训练数据中类别特征添加到对应列表中
        else:
            encoded_feature = L.Reshape((1,), name=f"{prefix}{feature_name}_reshape")(inputs[feature_name])
            num_features.append(encoded_feature)#将训练数据中数字特征添加到对应列表中

    features = (
        L.Concatenate(name=f"{prefix}inputs_concatenate")(cat_features + num_features)
        if concat_features
        else (cat_features, num_features)
    )#重塑数字特征以对输入进行编码

    return features

**Defining Model Configurations**
* Number of Outputs
* Activation of the Output Layer
* Number of Transformer Blocks
* Number of heads in the Transformer Blocks
* Embedding Dimension for the features
* Dimesion of the Dense Projections in the transfomer blocks

In [8]:
class FeatureTokenizerTransformerConfig:
    def __init__(
        self,
        num_outputs,#输出数量
        out_activation,#输出激活函数
        num_transformer_blocks=2,#transformer块的数量为2
        num_heads=8,#注意力头的数量为8
        embedding_dim=32,#类别型embedding嵌入的维度为32
        dense_dim=16,#隐藏层维度为16
    ):
        self.NUM_OUT = num_outputs#输出数量
        self.OUT_ACTIVATION = out_activation#输出激活函数
        self.NUM_TRANSFORMER_BLOCKS = num_transformer_blocks#transformer块的数量
        self.NUM_HEADS = num_heads#注意力头的数量
        self.EMBEDDING_DIM = embedding_dim#类别型embedding嵌入的维度
        self.DENSE_DIM = dense_dim#隐藏层维度

**Defining a standard Transformer Block**

In [9]:
class TransformerBlock(L.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim#字嵌入的维度
        self.dense_dim = dense_dim#隐藏层维度
        self.num_heads = num_heads#注意力头的数量
        self.attention = L.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)#多头注意力
        self.dense_proj = tf.keras.Sequential([L.Dense(dense_dim, activation="relu"), L.Dense(embed_dim)])#Sequential序列模型
        self.layernorm1 = L.LayerNormalization()#层标准化
        self.layernorm2 = L.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[: tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)#输出的多头注意力
        proj_input = self.layernorm1(inputs + attention_output)#将层标准化模型作为输入
        proj_output = self.dense_proj(proj_input)#将序列模型作为输出
        return self.layernorm2(proj_input + proj_output)

**Defining the Model**
The model takes Inputs Layers and then encodes the features from the functions defined above, the numerical features are then passed through a Dense layer of the same dimensions as the embeddings of the categorical features.

All the feature embeddings are then stacked and then passed through a series of Transformer Blocks followed by the Global Average Pooling and Final Output Layer

In [10]:
class FeatureTokenizerTransformer:
    @classmethod#类方法
    def from_config(cls, data_config, model_config, name):
        inputs = get_inputs(data_config)#获取输入
        cat_features, num_features = encode_inputs(
            inputs,#输入
            data_config,#数据配置
            use_embeddings=True,#使用嵌入向量
            embedding_dim=model_config.EMBEDDING_DIM,#嵌入向量的维度
            prefix="",#前缀初始为空
            concat_features=False,
        )
        num_feat_emb = [
            L.Dense(model_config.EMBEDDING_DIM, name=f"{feature_name}_embeddings")
            for _, feature_name in zip(range(len(num_features)), data_config.NUMERIC_FEATURE_NAMES)
        ]
        num_features = [emb(feat) for emb, feat in zip(num_feat_emb, num_features)]
        features = L.Concatenate(axis=1, name="feature_embeddings_stack")(
            [
                L.Reshape((1, 32), name=f"{feat_name}_reshape_2")(feat)
                for feat, feat_name in zip((num_features + cat_features), data_config.FEATURE_NAMES)
            ]
        )
        for _ in range(model_config.NUM_TRANSFORMER_BLOCKS):
            features = TransformerBlock(
                embed_dim=model_config.EMBEDDING_DIM,#字嵌入的维度
                dense_dim=model_config.DENSE_DIM,#隐藏层的维度
                num_heads=model_config.NUM_HEADS,#注意力头的数量
            )(features)
        features = L.GlobalMaxPooling1D()(features)#在steps维度对整个features数据求最大值
        outputs = L.Dense(#最终输出层
            units=model_config.NUM_OUT,#输出层
            activation=model_config.OUT_ACTIVATION,#输出激活函数
            name="outputs",#输出名称
        )(features)
        model = keras.Model(inputs=inputs, outputs=outputs, name=name)#模型
        return model

**Creating instances of the various classes defined so far**

In [11]:
data_config = DataConfig(#数据配置管理
    numeric_feature_names=X.columns.tolist(), categorical_features_with_vocabulary={}
)#配置数值特征及词汇特征
model_config = FeatureTokenizerTransformerConfig(num_outputs=len(y.columns), out_activation='softmax')#模型配置

In [12]:
blank_model = FeatureTokenizerTransformer.from_config(data_config, model_config, name='ftt')#建立空白模型
blank_model.summary()#函数打印空白模型摘要,其中包括模型名称、权重参数数量、可训练参数数量。

2022-03-31 14:05:31.911963: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-31 14:05:32.048224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-31 14:05:32.049350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-31 14:05:32.051022: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "ftt"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
A0T0G0C10 (InputLayer)          [(None,)]            0                                            
__________________________________________________________________________________________________
A0T0G1C9 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
A0T0G2C8 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
A0T0G3C7 (InputLayer)           [(None,)]            0                                            
________________________________________________________________________________________________

In [13]:
MAX_EPOCHS  = 50#最大运行的epoch数为50
get_callbacks = lambda : [
    keras.callbacks.EarlyStopping(min_delta=1e-4, patience=3, verbose=1, restore_best_weights=True),#使用早停法，防止模型过早拟合
    keras.callbacks.ReduceLROnPlateau(patience=2, verbose=1)#调整学习率，在训练过程中自动降低学习率
]